In [33]:
import subprocess
import re

In [55]:
# 读取代码文件
# path = '/root/wang/LLVM/mlir/test/Transforms/promote-buffers-to-stack.mlir'
# path = '/root/wang/LLVM/mlir/test/Dialect/SparseTensor/convert_sparse2dense.mlir'
path = '/root/wang/LLVM/mlir/test/Dialect/Linalg/canonicalize.mlir'
with open(path, "r") as file:
    code_content = file.read()

In [56]:
print(code_content)

// RUN: mlir-opt %s -canonicalize="test-convergence" -split-input-file | FileCheck %s

// CHECK-LABEL: func @memref_cast(
func.func @memref_cast(%a: index, %b: index) -> memref<?x?xf32> {
  %c0 = arith.constant 0 : index
  %c1 = arith.constant 1 : index
  %c8 = arith.constant 8 : index
  %c16 = arith.constant 16 : index
  %1 = memref.alloc (%b) : memref<?xi8>
  %2 = memref.view %1[%c0][] : memref<?xi8> to memref<16x16xf32>
  %3 = memref.cast %2 : memref<16x16xf32> to memref<?x?xf32>

  // CHECK:  linalg.matmul ins({{.*}}memref<16x16xf32>, memref<16x16xf32>) outs({{.*}}memref<16x16xf32>)
  linalg.matmul ins(%3, %3: memref<?x?xf32>, memref<?x?xf32>)
               outs(%3: memref<?x?xf32>)
  return %3: memref<?x?xf32>
}

// -----

#accesses = [
  affine_map<(i) -> (i)>
]

#trait = {
  indexing_maps = #accesses,
  iterator_types = ["parallel"]
}

func.func @dce_zero_memref(%arg0 : memref<0xf32>, %arg1: tensor<0xf32>) -> tensor<0xf32> {
  // memref<0x32> is expected to be dce'ed
  memref.c

In [34]:
def extract_code_body(code, start_marker, end_marker):
    start_idx = code.index(start_marker)
    end_idx = code.index(end_marker) + len(end_marker) 
    # print(start_idx, " ", end_idx)
    return code[start_idx:end_idx]

In [57]:
# 1. 提取正文内容
start_marker = "func.func @"
end_marker = "return\n}"

code_body = extract_code_body(code_content,start_marker,end_marker)

In [58]:
print(code_body)

func.func @memref_cast(%a: index, %b: index) -> memref<?x?xf32> {
  %c0 = arith.constant 0 : index
  %c1 = arith.constant 1 : index
  %c8 = arith.constant 8 : index
  %c16 = arith.constant 16 : index
  %1 = memref.alloc (%b) : memref<?xi8>
  %2 = memref.view %1[%c0][] : memref<?xi8> to memref<16x16xf32>
  %3 = memref.cast %2 : memref<16x16xf32> to memref<?x?xf32>

  // CHECK:  linalg.matmul ins({{.*}}memref<16x16xf32>, memref<16x16xf32>) outs({{.*}}memref<16x16xf32>)
  linalg.matmul ins(%3, %3: memref<?x?xf32>, memref<?x?xf32>)
               outs(%3: memref<?x?xf32>)
  return %3: memref<?x?xf32>
}

// -----

#accesses = [
  affine_map<(i) -> (i)>
]

#trait = {
  indexing_maps = #accesses,
  iterator_types = ["parallel"]
}

func.func @dce_zero_memref(%arg0 : memref<0xf32>, %arg1: tensor<0xf32>) -> tensor<0xf32> {
  // memref<0x32> is expected to be dce'ed
  memref.copy %arg0, %arg0 : memref<0xf32> to memref<0xf32>

  // tensor<0xf32> cannot be dce'ed
  %1 = linalg.generic #trait outs(%

In [78]:
pattern = r"(.*?)(?=// -----$|$)" 
statements = re.findall(pattern, code_content, re.DOTALL)


In [90]:

sections = re.split(r'\s*// -----\s*', code_content)
sections = [section.strip() for section in sections if section.strip() != ""]

for i, section in enumerate(sections, start=1):
    print(f"Section {i}:")

Section 1:
Section 2:
Section 3:
Section 4:
Section 5:
Section 6:
Section 7:
Section 8:
Section 9:
Section 10:
Section 11:
Section 12:
Section 13:
Section 14:
Section 15:
Section 16:
Section 17:
Section 18:
Section 19:
Section 20:
Section 21:
Section 22:
Section 23:
Section 24:
Section 25:
Section 26:
Section 27:
Section 28:
Section 29:
Section 30:
Section 31:
Section 32:
Section 33:
Section 34:
Section 35:
Section 36:
Section 37:


In [87]:
code_content.index("// -----")


729

In [81]:
print(statements[0])

In [65]:
statements = re.findall(r"// RUN:(.+?)\| FileCheck", code_content, re.DOTALL)


In [53]:
for i in range(len(statements)):
    statements[i] = statements[i].replace('\\\n// RUN:', '').replace('%s', '')

In [54]:
print(statements)

[' mlir-opt  --sparse-tensor-conversion --canonicalize --cse ', ' mlir-opt  --post-sparsification-rewrite="enable-runtime-library=false enable-foreach=false"  --canonicalize --cse ']


In [40]:
# 2. 查找pass
pass_command = extract_code_body(code_content,"mlir-opt","FileCheck")
pass_command = pass_command.replace('%s', '')
cleaned_pass = pass_command.rsplit(' | FileCheck', 1)[0]

In [41]:
print(cleaned_pass)

mlir-opt  --sparse-tensor-conversion --canonicalize --cse


In [1]:
for i in [1, 2, 3]:
    for j in [4, 5, 6]:
        print(i, j)

1 4
1 5
1 6
2 4
2 5
2 6
3 4
3 5
3 6


In [1]:
from extract import *
text = '''
// RUN: mlir-opt %s --run-reproducer -dump-pass-pipeline 2>&1 | FileCheck %s
// RUN: mlir-opt %s --run-reproducer -mlir-print-ir-before=cse 2>&1 | FileCheck -check-prefix=BEFORE %s
func.func @foo() {
  %0 = arith.constant 0 : i32
  return
}

func.func @bar() {
  return
}
'''
text = """
// RUN: mlir-opt %s --run-reproducer -dump-pass-pipeline 2>&1 | FileCheck %s
// RUN: mlir-opt %s --run-reproducer -mlir-print-ir-before=cse 2>&1 | FileCheck -check-prefix=BEFORE %s

func.func @foo() {
  %0 = arith.constant 0 : i32
  return
}

func.func @bar() {
  return
}

{-#
  external_resources: {
    mlir_reproducer: {
      verify_each: true,
      // CHECK:  builtin.module(func.func(cse,canonicalize{ max-iterations=1 max-num-rewrites=-1 region-simplify=false test-convergence=false top-down=false}))
      pipeline: "builtin.module(func.func(cse,canonicalize{max-iterations=1 max-num-rewrites=-1 region-simplify=false top-down=false}))",
      disable_threading: true
    }
  }
#-}

// BEFORE: // -----// IR Dump Before{{.*}}CSE (cse) //----- //
// BEFORE-NEXT: func @foo()
// BEFORE: // -----// IR Dump Before{{.*}}CSE (cse) //----- //
// BEFORE-NEXT: func @bar()
// BEFORE-NOT: // -----// IR Dump Before{{.*}}Canonicalizer (canonicalize) //----- //
// BEFORE-NOT: // -----// IR Dump After
"""
blocks = extract_block(text)
passes = extract_pass(text)
# 将新数据追加到 CSV 文件
with open(csv_filename, "a", newline="") as csv_file:
  csv_writer = csv.writer(csv_file)
  for b in blocks:
      for p in passes:
        csv_writer.writerow([b, p, default_cost])
        print(b, p)

func.func @foo() {
  %0 = arith.constant 0 : i32
  return
}
func.func @bar() {
  return
}
{-#
  external_resources: {
    mlir_reproducer: {
      verify_each: true,
      pipeline: "builtin.module(func.func(cse,canonicalize{max-iterations=1 max-num-rewrites=-1 region-simplify=false top-down=false}))",
      disable_threading: true
    }
  }
#-}  mlir-opt  --run-reproducer -dump-pass-pipeline 2>&1 
func.func @foo() {
  %0 = arith.constant 0 : i32
  return
}
func.func @bar() {
  return
}
{-#
  external_resources: {
    mlir_reproducer: {
      verify_each: true,
      pipeline: "builtin.module(func.func(cse,canonicalize{max-iterations=1 max-num-rewrites=-1 region-simplify=false top-down=false}))",
      disable_threading: true
    }
  }
#-}  mlir-opt  --run-reproducer -mlir-print-ir-before=cse 2>&1 
